In [1]:
import numpy as np
import sympy as sym
from pybullet_suite import *
from parse_urdf import *

pybullet build time: Nov  8 2022 14:29:22


In [2]:
#sympy
def get_sym_matrix(rpy=[0,0,0], trans=[0,0,0]):
    rot_mat = Rotation.from_euler("zyx", rpy[::-1]).as_matrix()
    rot_mat[np.abs(rot_mat) < 1e-9] = 0.
    T = np.block([[rot_mat, np.array(trans)[:,None]], [0,0,0,1]])
    return sym.Matrix(T)

# calculate forward kinematics
def get_T_lists(link_dict, joint_dict, qs, ee_link_name):
    T_link_list, T_joint_list = [], []
    link_name = ee_link_name  #"panda_grasptarget"
    while True:
        if "parent_joint" not in link_dict[link_name]:  break
        parent_joint_name = link_dict[link_name]["parent_joint"]
        parent_joint = joint_dict[parent_joint_name]
        T = get_sym_matrix(parent_joint["origin_rpy"], parent_joint["origin_xyz"])
        T_link_list.append(T)
        link_name = joint_dict[parent_joint_name]["parent"]
    T_link_list = T_link_list[::-1]

    i = 0
    while True:
        if "joint" not in link_dict[link_name]: break
        joint_name = link_dict[link_name]["joint"]
        if joint_dict[joint_name]["joint_type"] == "revolute":
            T_q = yaw_rot_mat(qs[i])
            T_joint_list.append(T_q)
        else:
            T_joint_list.append(None)
        link_name = joint_dict[joint_name]["child"]
        i += 1
    
    return T_link_list, T_joint_list



def yaw_rot_mat(q):
    s, c = sym.sin(q), sym.cos(q)
    return sym.Matrix([[c, -s, 0, 0],
                       [s, c, 0, 0],
                       [0, 0, 1, 0],
                       [0,0,0,1]])


def get_fk_sym_expr(link_dict, joint_dict, T_dict, root_link_name):
    link_name = root_link_name #"panda_link0"
    fk = sym.Matrix(np.eye(4))
    while True:
        if "joint" not in link_dict[link_name]: break
        joint_name = link_dict[link_name]["joint"]
        if joint_name in T_dict:
            fk *= T_dict[joint_name]
        if "q_"+joint_name in T_dict:
            fk *= T_dict["q_"+joint_name]
        link_name = joint_dict[joint_name]["child"]
    return fk

In [3]:
# Define the path to the URDF file
urdf_file = "panda.urdf"

# Parse the URDF file using ElementTree
link_dict, joint_dict = get_link_joint_dict(urdf_file)

Robot name: panda


In [4]:
q1, q2, q3, q4, q5, q6, q7 = sym.symbols('q1 q2 q3 q4 q5 q6 q7')
qs = [q1, q2, q3, q4, q5, q6, q7]
T_offset_list, T_joint_list = get_T_lists(link_dict, joint_dict, qs, "panda_grasptarget")

In [5]:
Ts = []
for T_offset, T_joint in zip(T_offset_list, T_joint_list):
    T = T_offset
    if T_joint is not None:
        T = T @ T_joint
    Ts.append(T)

In [23]:
#0 1
Ts[0]
# 1 2
Ts[1]
# 0 2
Ts[0] @ Ts[1]
def get_T(Ts, link_from, link_to):
    T = sym.Matrix(np.eye(4))
    for i in range(link_from, link_to):
        T = T@Ts[i]
    return T

get_T(Ts, 0, 3)[:3, -1]

Matrix([
[0.316*sin(q2)*cos(q1)],
[0.316*sin(q1)*sin(q2)],
[0.316*cos(q2) + 0.333]])

In [6]:
fk = sym.Matrix(np.eye(4))
for T in Ts:
    fk = fk @ T

In [7]:
# test
world = BulletWorld(gui=False)
panda = world.load_robot(robot_class=Panda, name="panda")

argv[0]=


In [8]:
q = panda.get_random_arm_angles()
q_values = {q:qval for q, qval in zip(qs, q)}
T_evaluated = fk.subs(q_values)

In [9]:
T_evaluated

Matrix([
[-0.553131911531489,  0.78992856415633,  0.264684627387838, 0.344082987256378],
[ 0.787427231061932, 0.599468195862772, -0.143513894557885, 0.287905764579648],
[-0.272035740717391, 0.129037768420533, -0.953596251090255, 0.578119764170826],
[                 0,                 0,                  0,               1.0]])

In [11]:
panda.forward_kinematics(q).as_matrix()

array([[-0.55313191,  0.78992856,  0.26468463,  0.34408299],
       [ 0.78742723,  0.5994682 , -0.14351389,  0.28790576],
       [-0.27203574,  0.12903777, -0.95359625,  0.57811976],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [18]:
len(T_joint_list)

9

In [10]:
T_link_list[0]

Matrix([
[1.0,   0,   0,     0],
[  0, 1.0,   0,     0],
[  0,   0, 1.0, 0.333],
[  0,   0,   0,   1.0]])

In [11]:
T_joint_list[0]

Matrix([
[cos(q1), -sin(q1), 0, 0],
[sin(q1),  cos(q1), 0, 0],
[      0,        0, 1, 0],
[      0,        0, 0, 1]])

In [ ]:
T_dict = get_T_dict(link_dict, joint_dict, qs)
fk = get_fk_sym_expr(link_dict, joint_dict, T_dict, "panda_link0")

In [ ]:
# test
world = BulletWorld(gui=False)
panda = world.load_robot(Panda, "panda")
q = panda.get_random_arm_angles()
q_values = {q:qval for q, qval in zip(qs, q)}
T_evaluated = fk.subs(q_values)